In [15]:
import requests  
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import mysql.connector as SQLC
import mysql.connector
from datetime import date
import time
from bs4 import BeautifulSoup
from Bio.SeqFeature import CompoundLocation
from datetime import datetime

In [46]:
#Extrair ids genebank:
data_e_hora_atuais = datetime.now()

query= input('escolha o que quer pesquisar: ')

def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
    
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
    

existe = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(existe)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20): '))]
numero_genes= len(n_genes)

#Extrair ids ncbi:

Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)

#extrair links ncbi
def url_get_id(i):
    url_list= [ ]
    for id in i:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
        url_list.append(url)
    return url_list
link_genebank= url_get_id(Ids)
print(link_genebank)

#Extrair description NCBI:
description=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    description.append(info.description)

#organismos

Organismos=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Organismos.append(info.annotations['organism'])
    
#SEQ
seq=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    seq.append(f'{info.seq[0:10]}...{info.seq[-10:]}')

#percentagem de nucle:
Adenina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('A')
    perc= int(count/len(c)*100)
    Adenina.append(perc)

Citosina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('C')
    perc= int(count/len(c)*100)
    Citosina.append(perc)

Guanina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('G')
    perc= int(count/len(c)*100)
    Guanina.append(perc)

Timina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('T')
    perc= int(count/len(c)*100)
    Timina.append(perc)
    
#data
data=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    data.append(info.annotations['date'])

#len(SEQ)
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
tam=[]
for info in records:
    tam.append(len(info.seq))

# buscar info pubmed
content_id = []
for url in url_get_id(Ids):
    r_id = requests.get(url)
    content_id.append(r_id.content)
    
from bs4 import BeautifulSoup
listas=[]

for c in content_id:
    soup_id = BeautifulSoup(c, 'html.parser')

    lines = soup_id.find_all('meta', {'name':"ncbi_uidlist"} )

    id = ""
    url = ""
    for line in lines:
        if 'content' in line.attrs:
            id = line.attrs['content']
    if id:
        url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id)
    r2 = requests.get(url)
    r4= r2.content.decode('utf-8')
    listas.append(r4)
    
cc= ', '.join(listas)
er= cc.replace('//','')
final= er.split(', ')

#criar dicionário de ids ncbi e ids pubmed
output_dict = {}
for x in final:
    version = re.search(r'VERSION\s+(.*?)\s', x)
    pubmed = re.search(r'PUBMED\s+(.*?)\s', x)
    if version:
        versionf=version.group(1)
        output_dict.setdefault(version.group(1), [])
    if pubmed:
        output_dict[versionf].append(pubmed.group(1))
#     else:
       # output_dict[version.group(1)].append("N/A")

#Criar Listas só com ids ncbi e ids pubmed
id_ncbii = []
ID_PUB = []
for key, vals in output_dict.items():
    if vals:
        for val in vals:
            id_ncbii.append(key)
            ID_PUB.append(val)
    else:
        id_ncbii.append(key)
        ID_PUB.append("N/A")

new_list_ = []
seen = set()
for item in ID_PUB:
    if item not in seen:
        seen.add(item)
        new_list_.append(item)

so_ids_pubmed = [val for key, vals in output_dict.items() for val in vals]
so_ids_ncbi = list(output_dict.keys())


escolha o que quer pesquisar: virus
escolha o nº de genes que quer obter (máximo 20): 5
['https://www.ncbi.nlm.nih.gov/nuccore/V01302.1', 'https://www.ncbi.nlm.nih.gov/nuccore/X67705.1']


In [47]:
#Extrair informação de artigos

titles=[]
authors=[]
source=[]
affiliation=[]
database = 'PubMed'
email= 'rodrigoce9@gmail.com'
idlist= new_list_
counter = 0
for i in idlist:
    if i!= "N/A":
        handle = Entrez.efetch(db=database, id=i, rettype="medline", retmode="text") 
        records = Medline.parse(handle)
        for info in records:
            titles.append(info.get("TI", ["N/A"]))
            authors_string = info.get("AU", ["N/A"])
            if len(authors_string) > 5:
                authors_h = authors_string[0:5]
                authors.append(authors_h)
            else:
                authors.append(authors_string) 
            source.append(info.get("SO", ["N/A"]))
            affiliation_string= info.get("AD", ["N/A"])
            if len(affiliation_string) > 5:
                affiliation_h = affiliation_string[0:5]
                affiliation.append(affiliation_h)
            else:
                affiliation.append(affiliation_string)
            counter += 1
    else:
        titles.append(["N/A"])
        authors.append(["N/A"])
        source.append(["N/A"])
        affiliation.append(["N/A"])
        counter += 1
        
#agrupar titles
titles = [ [title] for title in titles]

# agrupar dois
doi_list = []
for x in source:
    match = re.search("doi: (.*)", str(x))
    if match:
        doi_list.append(match.group(1))
    else:
        doi_list.append("N/A")
        
#agrupar authors
id_authors_dict = {i: authors[counter] if i != "N/A" else ["N/A"] for counter, i in enumerate(idlist)}        
        
pubmed_list = []
authors_list = []
for key, vals in id_authors_dict.items():
    if vals:
        for val in vals:
            pubmed_list.append(key)
            authors_list.append(val)

new_list_authors= []
seen = set()
for item in authors_list:
    if item not in seen:
        seen.add(item)
        new_list_authors.append(item)

#agrupar affi
single_affiliation_list = []
for i in affiliation:
    single_affiliation_list.extend(i)
    

id_affiliation_dict = {i: [single_affiliation_list[counter]] if i != "N/A" else ["N/A"] for counter, i in enumerate(idlist)}

pubmed_affi_list = []
affi_list = []
for key, vals in id_affiliation_dict.items():
    if vals:
        for val in vals:
            pubmed_affi_list.append(key)
            affi_list.append(val)
    else:
        pubmed_affi_list.append(key)
        affi_list.append(["N/A"])

new_list_affi= []
seen = set()
for item in single_affiliation_list :
    if item not in seen:
        seen.add(item)
        new_list_affi.append(item)
        
#FINAL para protein_id - PROT_ID (NECESSÁRIO)
list_pro=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    for i in info.features:
        if i.type == "CDS":
            pro= str(i.qualifiers["protein_id"])
            list_pro.append(pro)
div= ', '.join(list_pro)
h= div.replace("['",'')
hh= h.replace("']",'')
ID_PROT= hh.split(', ')
print (ID_PROT) # is used below in Uniprot

#FINAL para location, product e translation  - get_CDS_info(Ids) = [id_protein, location, product, translation] - NECESSÁRIO
def get_CDS_info(id_list):
    database = 'nucleotide'
    email= 'rodrigoce9@gmail.com'
    Entrez.email = email
    handle = Entrez.efetch(db=database, id=id_list, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    cds_location_list = []
    
    several_location = []
    record_types={}
    for i in id_list:
        handle2 = Entrez.efetch(db=database, id=i, rettype="gb")
        for record in SeqIO.parse(handle2, "genbank"):
            for feature in record.features:
                record_types[feature.type]=record_types.get(feature.type,0)+1
                cds_nonlocation, nonlocation, nonproduct, nontranslation = [], [], [], []
            if 'CDS' not in record_types:
                cds_nonlocation.append("N/A_CDS")
                nonlocation.append("N/A")
                nonproduct.append("N/A")
                nontranslation.append("N/A")
                cds_nonlocation.append(nonlocation)
                cds_nonlocation.append(nonproduct)
                cds_nonlocation.append(nontranslation)
                cds_location_list.append(cds_nonlocation)
#                 cds_location_list.append(nonlocation)
                
    handle2.close()
    for info in records:
        for i in info.features:
            list_product, list_translation = [],[]
            cds_location = []
            product, translation = '', ''
            if i.type == "CDS":
                i_d = str(i.qualifiers["protein_id"])
                product = str(i.qualifiers["product"])
                translation =  str(i.qualifiers["translation"])
                cds_location.append(i_d)
                if isinstance(i.location, CompoundLocation):
                    for sub_location in i.location.parts:
                        several_location.append("[{} : {}]".format(sub_location.start, sub_location.end))
                    cds_location.append(several_location)
                    cds_location.append(product)
                    cds_location.append(translation)
                else:
                    cds_location.append("[{} : {}]".format(i.location.start, i.location.end))
                    cds_location.append(product)
                    cds_location.append(translation)
                cds_location_list.append(cds_location)      
    handle2.close()                    
    return cds_location_list
print(get_CDS_info(Ids))

#FINAL result_dic = {Ids, ID_PROT} - NECESSÁRIO
result_dict = {}
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist=Ids
# idlist= Ids.remove('U49845')
for ids in idlist:
    list_pro=[]
    handle = Entrez.efetch(db=database, id=ids, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    for info in records:
        list_pro.append(info.id)
        for i in info.features:
            if i.type == "CDS":
                pro= str(i.qualifiers["protein_id"])
                list_pro.append(pro)
        if len(list_pro)==1: #if no protein_id was found
            result_dict[info.id] = "N/A_CDS"
        else:
            list_pro = [x.replace("['",'').replace("']",'') for x in list_pro]
            result_dict[info.id] = ', '.join(list_pro[1:])
print(result_dict)
#key: id_genebank; values: id_Uniprot

#final, mas não correr (código necessário para o caso se querermos links não diretos para uniprot )
def url_get_id_p(dic):
    url_list_p = []
    for key, value in dic.items():
        if value != 'N/A_CDS':
            value_list = value.split(',')
            for val in value_list:
                val = val.strip()
                url_id_p = "https://www.uniprot.org/uniprotkb?query={}".format(val)
                url_list_p.append(url_id_p)
    return url_list_p
url_ids_protein=url_get_id_p(result_dict)
print(url_ids_protein) # but we don't need to save this links

#FINAL (usado em baixo) - NECESSÁRIO
def url_get_id_p(string):
#     print(string)
    url_list_p=[ ]
    url_id_p= "https://www.uniprot.org/uniprotkb/{}/entry".format(str(string))
    url_list_p.append(url_id_p)
    return ''.join(url_list_p)
print(url_get_id_p)


#FINAL - NECESSÁRIO
WEBSITE_API = "https://rest.uniprot.org"
fields = ["accession","organism_name","protein_name","cc_subcellular_location","cc_function", "sequence"]
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);
    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()
    return response

def get_field_for_id(ID_PROT, field):
    response = get_url("{}/uniprotkb/search?query={}&fields={}&size=1&format=tsv".format(WEBSITE_API,ID_PROT,field))
    return str(response.content)

def get_list_uniprot(ID_PROT, result_dict):
    results = {}
    result = {}
    for first_key, first_value in result_dict.items(): 
        result = {}
        for field in fields:
            if first_value != 'N/A_CDS':
                result[field] = get_field_for_id(first_value, field)
            else:
                result[field] = 'N/A'
            results[first_key] = result   
    
    uniprot_final_list=[]
    for key, value in results.items():
        uniprot_list=[]
        for x in value.values():
#             print(key)
            entry = re.search(r'b\'Entry\\n(.+?(?=\\n\'))', x, re.DOTALL)
            function = re.match( r'b\'Function \[CC\]\\n.{9} (.+?(?=\\n\'))', x, re.DOTALL )
            location_exist = re.search( r'b\'Subcellular location \[CC\]\\nSUBCELLULAR LOCATION: (.+?(?=\\n\'))', x, re.DOTALL )
            location_notexist = re.search( r'b\'Subcellular location \[CC\]\\n\\n\'', x, re.DOTALL )   
            sequence = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', x, re.DOTALL)
            n_a = re.search(r'(N/A)', x, re.DOTALL)
            if entry:
                ent=entry.group(1)
                uniprot_list.append(entry.group(1))            
            if function:
                uniprot_list.append(function.group(1))
#                 print(function.group(0))
#             if not function:
#                 print('na')
            if location_exist:
                uniprot_list.append(location_exist.group(1))
            if location_notexist:
                uniprot_list.append("N/A")
            if sequence:
                uniprot_list.append(sequence.group(1))
                uniprot_list.append(len(sequence.group(1)))
                uniprot_list.append(url_get_id_p(ent))
            if n_a:
                uniprot_list.append(n_a.group(1))
                
        uniprot_final_list.append(uniprot_list)
        
    return uniprot_final_list
print(get_list_uniprot(ID_PROT,result_dict)) #list of lists (ID_Uniprot, Function, Subcelular location, Protein seq, length_aa)

#FINAL [idgenebank, protein_id, id do uniprot]  -  NECESSÁRIO
def join_ids_CDS(dic, uniprotID):
    join_list_all = []
    for key, value in dic.items():
        join_list = []
        join_list.append(key)
        join_list.append(value)
        join_list_all.append(join_list) 
    for index, values in enumerate(uniprotID):
        join_list_all[index].append(uniprotID[index][0])
    return join_list_all
join_ids_CDS(result_dict,get_list_uniprot(ID_PROT, result_dict))

#final [idgenebank, protein_id, id_uniprot,location, product, translation] (NÃO SEI SE ESTÁ A FUNCAR DIREITO - assumir que não)
def join_lists(list1, list2):
    result = []
    for item1 in list1:
        print(item1)
        for item2 in list2:
            print(item2)
            string_item2 = str(item2[0])
            string_item2 = string_item2.replace("[","").replace("]","").replace("'","")
            if item1[1] == string_item2 or item1[1] == string_item2:
                result.append(item1 + item2[1:])
    return result
join_lists(join_ids_CDS(result_dict,get_list_uniprot(ID_PROT, result_dict)), get_CDS_info(Ids))



['CAA24609.1', 'CAA47943.1']


KeyError: 'product'

In [48]:
#Povoação "History"

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "INSERT INTO History (search, Genes_number_input, Day)   VALUES (%s, %s, %s)"
    val=(query, numero_genes, data_e_hora_atuais)
    Cursor.execute(sql,val)
    
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
search_id =[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "Select ID_search FROM History"
    Cursor.execute(sql)
    for row in Cursor:
        search_id.append(str(row)) 
        
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div= ', '.join(search_id)
h= div.replace("(",'')
hh= h.replace(",)",'')
SEARCH_ID= hh.split(', ')
Hist= SEARCH_ID[-1]

#Povoação "Gene"

#tab = ("INSERT INTO Gene (ID_genebank) VALUES ('{}')".format(value)) # %s overwrites the same value

des = (description)
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(Ids):
        #print(value)
        #print(des[index])
        sql= "INSERT INTO Gene (ID_genebank, Description, Organism, length, Adenina, Citosina, Guanina, Timina, Link, ID_search) VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val=(value, des[index], Organismos[index], tam[index], Adenina[index], Citosina[index], Guanina[index], Timina[index], link_genebank[index], Hist)
#         print(type(des[index]))
        Cursor.execute(sql,val)
#         for values in des:
#             Cursor.execute(f"INSERT INTO Gene ( Description)  VALUES ('{values}')")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
#Povoar "PubMed"

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(new_list_):
        #print(value)
        #print(des[index])
        sql= "INSERT INTO PubMed (ID_PumMed, title, Doi_number) VALUES (%s, %s, %s)"
        val=(str(new_list_[index]), str(titles[index]), str(doi_list[index]) )
#         print(type(des[index]))
        Cursor.execute(sql,val)
#         for values in des:
#             Cursor.execute(f"INSERT INTO Gene ( Description)  VALUES ('{values}')")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
#retirar os valores AI de pubmed
ID_AI=[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "Select ID_AI_PubMed FROM PubMed"
    Cursor.execute(sql)
    for row in Cursor:
        #print(row)
        ID_AI.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
   
div_= ', '.join(ID_AI)
h_= div_.replace("(",'')
hh_= h_.replace(",)",'')
SEARCH_ID_= hh_.split(', ')
#print(SEARCH_ID_)

#Povação "Gene-Pubmed"
number_map = {}
next_number = int(SEARCH_ID_[0])
relation = []
for number in ID_PUB:
    if number not in number_map:
        number_map[number] = next_number
        next_number += 1
    relation.append(number_map[number])
    

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(id_ncbii):
        
        sql= "INSERT INTO gene_PubMed (ID_genebank, ID_AI_PubMed) VALUES (%s, %s)"
        val=(id_ncbii[index], relation[index])
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
#Povação "authors"
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(new_list_authors):
        
        sql= "INSERT INTO Authors (Name) VALUES (%s)"
        val=(new_list_authors[index],)
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

#retirar os valores AI de authors
ID_AI_Authors=[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "Select ID_Authors FROM Authors"
    Cursor.execute(sql)
    for row in Cursor:
        #print(row)
        ID_AI_Authors.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div_Authors= ', '.join(ID_AI_Authors)
h_Authors= div_Authors.replace("(",'')
hh_Authors= h_Authors.replace(",)",'')
SEARCH_ID_Authors= hh_Authors.split(', ')
#print(SEARCH_ID_Authors)

#Povação "Pubmed-Authors"
number_map_ = {}
next_number = int(SEARCH_ID_Authors[0])
output_authors = []
for number in authors_list:
    if number not in number_map_:
        number_map_[number] = next_number
        next_number += 1
    output_authors.append(number_map_[number])

number_map_pub = {}
next_number = int(SEARCH_ID_[0])
output_pub = []
for number in pubmed_list:
    if number not in number_map_pub:
        number_map_pub[number] = next_number
        next_number += 1
    output_pub.append(number_map_pub[number])

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(output_pub):
        
        sql= "INSERT INTO PubMed_Authors (ID_AI_PubMed, ID_Authors) VALUES (%s, %s)"
        val=(str(output_pub[index]),str(output_authors[index]))
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
#Povação "affi"
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(new_list_affi):
        
        sql= "INSERT INTO Affiliation (Info) VALUES (%s)"
        val=(new_list_affi[index],)
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

    
#retirar os valores AI de affi
ID_AI_Affiliation=[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "Select ID_Affiliation FROM Affiliation"
    Cursor.execute(sql)
    for row in Cursor:
        #print(row)
        ID_AI_Affiliation.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div_Affiliation= ', '.join(ID_AI_Affiliation)
h_Affiliation= div_Affiliation.replace("(",'')
hh_Affiliation= h_Affiliation.replace(",)",'')
SEARCH_ID_Affiliation= hh_Affiliation.split(', ')
#print(SEARCH_ID_Affiliation)

#Povação "Pubmed-Affiliation"
number_map_affi_pub = {}
next_number = int(SEARCH_ID_[0])
output_pub_affi = []
for number in pubmed_affi_list:
    if number not in number_map_affi_pub:
        number_map_affi_pub[number] = next_number
        next_number += 1
    output_pub_affi.append(number_map_affi_pub[number])
    
number_map_affi = {}
next_number = int(SEARCH_ID_Affiliation[0])
output_affi = []
for number in affi_list:
    if number not in number_map_affi:
        number_map_affi[number] = next_number
        next_number += 1
    output_affi.append(number_map_affi[number])


DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(output_pub_affi):
        
        sql= "INSERT INTO PubMed_Affiliation (ID_AI_PubMed, ID_Affiliation) VALUES (%s, %s)"
        val=(str(output_pub_affi[index]),str(output_affi[index]))
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()



Erro na escrita na base de dados: 1062 (23000): Duplicate entry '91-310' for key 'PRIMARY'
Erro na escrita na base de dados: 1062 (23000): Duplicate entry '91-57' for key 'PRIMARY'


In [32]:
#Povoação "CDS"

import mysql.connector as SQLC
import mysql.connector


DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(Ids):
        #print(value)
        #print(des[index])
        sql= "INSERT INTO CDS (ID_CDS, Product, Translation, Location, ID_genebank, ID_Uniprot) VALUES (%s, %s, %s, %s, %s, %s)"
        val=(get_CDS_info(Ids)[index][0], get_CDS_info(Ids)[index][2], get_CDS_info(Ids)[index][3],get_CDS_info(Ids)[index][1],join_ids_CDS(result_dict,get_list_uniprot(ID_PROT, result_dict))[index][0],join_ids_CDS(result_dict,get_list_uniprot(ID_PROT, result_dict))[index][2])
#         print(type(des[index]))
        Cursor.execute(sql,val)
#         for values in des:
#             Cursor.execute(f"INSERT INTO Gene ( Description)  VALUES ('{values}')")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    

#Povoação "Uniprot"
# print(uniprot_final_list) #(ID_Uniprot, Subcelular location, Function, Protein seq, length_aa)

import mysql.connector as SQLC
import mysql.connector


DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
#     sql = "ALTER TABLE Uniprot AUTO_INCREMENT = 0"
#     Cursor.execute(sql)
    for index, value in enumerate(get_list_uniprot(ID_PROT)):
        #print(value)
        sql= "INSERT INTO Uniprot (ID_Uniprot, Subcelular_Location, Function, Protein_sequence, length_aa, Link_Uniprot) VALUES (%s, %s, %s, %s, %s, %s)"
        val=(get_list_uniprot(ID_PROT)[index][0],get_list_uniprot(ID_PROT)[index][1], get_list_uniprot(ID_PROT)[index][2], get_list_uniprot(ID_PROT)[index][3], get_list_uniprot(ID_PROT)[index][4], get_list_uniprot(ID_PROT)[index][5])
        Cursor.execute(sql,val)
    
    ID_AI=[]
    sql= "Select ID_AI FROM Uniprot"
    Cursor.execute(sql)
    for row in Cursor:
        ID_AI.append(str(row))
    div_= ', '.join(ID_AI)
    h_= div_.replace("(",'')
    hh_= h_.replace(",)",'')
    SEARCH_ID_AI= hh_.split(', ')
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

#para buscar os ids da tabela Uniprot, que sao AI
ID_AI=[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "Select ID_AI FROM Uniprot"
    Cursor.execute(sql)
    for row in Cursor:
        #print(row)
        ID_AI.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div_= ', '.join(ID_AI)
h_= div_.replace("(",'')
hh_= h_.replace(",)",'')
SEARCH_ID_AI= hh_.split(', ')
print(SEARCH_ID_AI)

Erro na escrita na base de dados: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`AP_db_KRG`.`CDS`, CONSTRAINT `uniprot` FOREIGN KEY (`ID_Uniprot`) REFERENCES `Uniprot` (`ID_Uniprot`) ON DELETE NO ACTION ON UPDATE NO ACTION)


TypeError: get_list_uniprot() missing 1 required positional argument: 'result_dict'

In [39]:
import mysql.connector as SQLC
import mysql.connector

des = (description)
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
        
        a= "delete from PubMed_Affiliation"
        Z= "delete from PubMed_Authors"
        b= "delete from Affiliation"
        c= "delete from Authors"
        d= "delete from gene_PubMed"
        e= "delete from PubMed"
        f= "delete from CDS"
        g= "delete from Uniprot"
        h= "delete from Gene"
        i= "delete from History"     
        
        Cursor.execute(a)
        Cursor.execute(Z)
        Cursor.execute(b)
        Cursor.execute(c)
        Cursor.execute(d)
        Cursor.execute(e)
        Cursor.execute(f)
        Cursor.execute(g)
        Cursor.execute(h)
        Cursor.execute(i)
        
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()